# Yijing Oracle Development Notebook

Dieses Notebook dient der systematischen Entwicklung und dem Testing des Yijing Oracle Moduls. Wir werden Schritt für Schritt die Komponenten testen und verfeinern.

## Inhaltsverzeichnis
1. Setup und Grundkonfiguration
2. Basismodelle testen (Linien, Hexagramme)
3. Orakel-Funktionalität testen
4. Integration mit GenAI API
5. End-to-End Tests

Legen wir los!

## 1. Setup und Grundkonfiguration

Zuerst importieren wir die notwendigen Module und setzen die Grundkonfiguration.

In [ ]:
import os
import sys
from pathlib import Path
import logging
import json

from yijing.managers import HexagramManager
from IPython.display import display, Markdown

# Füge den Projektordner zum Python-Path hinzu
project_dir = Path.cwd().parent  # Passe dies an deine Struktur an
sys.path.append(str(project_dir))

# Konfiguriere Logging
logging.basicConfig(level=logging.DEBUG)

# Importiere das yijing Modul
from yijing.models import HypergramLine, Hexagram, Hypergram
from yijing.hypergram import cast_hypergram
from yijing.oracle import YijingOracle
from yijing.enums import ConsultationMode

print("Setup erfolgreich abgeschlossen!")

In [ ]:
# Initialize resources directory
resources_dir = Path.cwd() / 'yijing' / 'resources'

# Create manager instance
manager = HexagramManager(resources_dir)

# Test code for the notebook
try:
    # Create reading context
    context = manager.create_reading_context(
        original_hex_num=1,
        changing_lines=[3, 6],
        resulting_hex_num=2
    )
    
    # Generate consultation prompt
    test_prompt = manager.get_consultation_prompt(
        context=context,
        question="Was sollte ich in meiner aktuellen Situation beachten?"
    )
    
    #print(test_prompt[:400])
    display(Markdown(test_prompt))

except Exception as e:
    print(f"Fehler beim Testen: {e}")

## 2. Basismodelle testen

Hier testen wir die grundlegenden Modelle wie HypergramLine, Hexagram etc.

In [ ]:
# Test: HypergramLine erstellen und Eigenschaften prüfen
def test_hypergram_line():
    # Teste verschiedene Linientypen
    lines = [
        HypergramLine(value=6),  # Changing Yin
        HypergramLine(value=7),  # Stable Yang
        HypergramLine(value=8),  # Stable Yin
        HypergramLine(value=9)   # Changing Yang
    ]
    
    for line in lines:
        print(f"\nLine value: {line.value}")
        print(f"Is Yin: {line.is_yin()}")
        print(f"Is Yang: {line.is_yang()}")
        print(f"Is Changing: {line.is_changing()}")
        print(f"Unicode Symbol: {line.to_unicode_symbol()}")

test_hypergram_line()

## 3. Orakel-Funktionalität testen

Jetzt testen wir die Hauptfunktionalität des Orakels.

In [ ]:
# Neue Zelle im Workbook

import os
import json
from pathlib import Path
from typing import Dict, Any, Optional
from functools import lru_cache

# Projektverzeichnis bestimmen
project_dir = Path.cwd()#.parent  # Ein Verzeichnis höher, da wir im workbook/ sind
resources_dir = project_dir / 'yijing' / 'resources'

@lru_cache(maxsize=64)  # Caching für bessere Performance
def load_hexagram_data(hexagram_number: int) -> Dict[str, Any]:
    """
    Lädt die JSON-Daten für ein spezifisches Hexagramm.
    
    Args:
        hexagram_number: Nummer des Hexagramms (1-64)
        
    Returns:
        Dict mit den Hexagramm-Daten
        
    Raises:
        FileNotFoundError: Wenn die Hexagramm-Datei nicht gefunden wird
    """
    hexagram_file = resources_dir / 'hexagram_json' / f'hexagram_{hexagram_number:02d}.json'
    
    if not hexagram_file.exists():
        raise FileNotFoundError(f"Hexagramm-Datei nicht gefunden: {hexagram_file}")
        
    with open(hexagram_file, 'r', encoding='utf-8') as f:
        return json.load(f)

# Test der Funktion und Verzeichnisstruktur
print(f"Projektverzeichnis: {project_dir}")
print(f"Resources Verzeichnis: {resources_dir}")
print(f"Resources Verzeichnis existiert: {resources_dir.exists()}")

try:
    hexagram_1 = load_hexagram_data(1)
    print(f"\nHexagramm 1 geladen: {hexagram_1['hexagram']['name']}")
except Exception as e:
    print(f"\nFehler beim Laden: {e}")

In [ ]:
# API Key setzen (ersetze dies durch deinen tatsächlichen Key)
api_key = os.environ["API_KEY"]

# Oracle initialisieren
oracle = YijingOracle()

# Test: Einfache Anfrage
response = oracle.get_response("Was sollte ich heute beachten?")
print("Oracle Antwort:")
print(response['answer'])

## 4. Integration mit GenAI API

Hier testen wir die Integration mit der Google GenAI API.

In [ ]:
# Test: Dialog-Modus
oracle_dialogue = YijingOracle(
    custom_settings={
        "consultation_mode": ConsultationMode.DIALOGUE
    }
)

# Starte eine neue Beratung
oracle_dialogue.start_new_consultation()
response = oracle_dialogue.get_response("Wie kann ich meine aktuelle Situation verbessern?")
print("Dialog-Antwort:")
print(response['answer'])

## 5. End-to-End Tests

Abschließend führen wir einige End-to-End Tests durch.

In [ ]:
def run_end_to_end_test():
    print("Starting End-to-End Test...\n")
    
    # 1. Hypergram werfen
    hypergram_data = cast_hypergram()
    print("Geworfenes Hypergram:")
    print(f"Altes Hexagramm: {hypergram_data.old_hexagram.to_unicode_representation()}")
    print(f"Neues Hexagramm: {hypergram_data.new_hexagram.to_unicode_representation()}")
    print(f"Wandelnde Linien: {hypergram_data.changing_lines}\n")
    
    # 2. Oracle Antwort generieren
    response = oracle.get_response("Wie soll ich mit Veränderungen umgehen?")
    print("Oracle Antwort:")
    print(response['answer'])

run_end_to_end_test()

## Nächste Schritte

1. Fehlerbehandlung verbessern
2. Tests für Edge Cases hinzufügen
3. Dokumentation erweitern
4. Performance optimieren

Notiere hier Beobachtungen und nötige Anpassungen: